In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import yfinance as yf

from yahoo_fin import stock_info as si

from pandas_datareader import data

#ignore warnings 

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('cleaned_data.csv')

In [3]:
df.head()

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,marketcap_cat,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,NaN,NaN,NaN,NaN,214.150,0.103398,1.308405e+10,NaN,1.25,9.550,0.1525,13.932986,22.424082,NaN,12.97,NaN
1,ABT,Abbott Laboratories,Healthcare,NaN,NaN,NaN,NaN,118.290,0.181179,2.091710e+11,NaN,0.74,3.940,0.1318,22.924420,30.022842,NaN,25.96,NaN
2,ACN,Accenture plc,Technology,NaN,NaN,NaN,NaN,323.905,0.208836,2.047083e+11,NaN,1.21,9.613,0.1135,31.025385,33.694477,NaN,9.45,NaN
3,ADM,Archer-Daniels-Midland Company,Consumer Defensive,NaN,NaN,NaN,NaN,76.985,0.122815,4.327843e+10,NaN,0.81,4.790,0.0660,14.804809,16.072025,NaN,36.42,NaN
4,ADP,"Automatic Data Processing, Inc.",Industrials,NaN,NaN,NaN,NaN,201.010,0.182708,8.443324e+10,NaN,0.83,6.460,0.1371,26.483530,31.116098,NaN,14.18,NaN


In [4]:
def cumm_prod(price_df, start=None, end=None):
    """
    Calculating the cummulative product of the percentage change to normalize the return and be
    able to compare the stocks. 

    """
    if start == None and end == None:
        return_series = (price_df.pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
    else:
        if end == None:
            end = date.today()
        return_series = (price_df.loc[start:end].pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
        
    
    return return_series

def annualized_return(adjClose_return_df, n_years=None):
    """
    n_years needs to represent the period of the adjClose_return_df.
        If it's 1 year worth of cummulative return, you input 1 for the 1y annualized returns.
        If it's 3 years worth of cummulative return, you input 3 for the long period annualized returns.

        (1+total return)pow(1/N) - 1) where N = number of years
    """
    if n_years == None or n_years == 0:
        print("Input How many years does your data covers!")
    else:
        annualized_return = ((1 + adjClose_return_df.tail(1))**(1/n_years)-1) * 100
        return annualized_return

In [5]:
def market_cap_cat(market_cap):
    try:
        if market_cap > 10000000000:
            market_cap_cat = 'Large'
        elif 2000000000 < market_cap < 1000000000:
            market_cap_cat = 'Medium'
        else:
            market_cap_cat = 'Small'
        return market_cap_cat
    except:
        return None

def earnings_expectations(FPE, TPE):
    try:
        if FPE == None or TPE == None:
            return None
        else:
            if FPE > TPE:
                expect = 'Decrease'
            else:
                expect = 'Increase'
                
            return expect
    except:
        return None

def CAPM_analysis(beta, annual_return, market_return, risk_free_rate = 0.0192):
    try:
        if beta == None or annual_return == None or market_return == None:
            return None
        else:
            capm_status = 'TBD'
            
            capm_ER = risk_free_rate + beta * (market_return - risk_free_rate)

            if annual_return > capm_ER:
                capm_status = 'OverValued'
            else:
                capm_status = 'UnderValued'
        
        return capm_status, capm_ER
    except:
        return None
        
def intrinsic_valuation(ticker, eps_ttm, growth_estimate_5y, PE_trailing, target_return, safety_margin=0.5):

    try:
        if ticker == None or eps_ttm == None or growth_estimate_5y == None or PE_trailing == None or target_return == None:
            return None
        else:
            irv_status = 'TBD'

            forecast_eps_at_year_10 = eps_ttm * ( 1 + growth_estimate_5y) ** 9
            forecast_value_at_year_10 = forecast_eps_at_year_10 * PE_trailing

            pv_future_value = forecast_value_at_year_10/(1 + target_return)**9

            pv_forecast_value_safe = pv_future_value * safety_margin

            if float(si.get_live_price(ticker)) > pv_forecast_value_safe:
                irv_status = 'Above Fair Price'
            else:
                irv_status = 'Below Fair Price'

            return irv_status, pv_forecast_value_safe
    except:
        return None

def get_ESG(ticker):
    ticket = yf.Ticker(ticker)
    try:
        esg_data = pd.DataFrame(ticket.sustainability)['Value']['totalEsg']
        return esg_data
    except:
        pass

def esg_status(esg_Rscore):
    if esg_Rscore == 999:
        return 'No Data'
    else:
        if float(esg_Rscore) < 20:
            return 'Sustainable'
        elif 21 < float(esg_Rscore) < 30:
            return 'Average'
        else:
            return 'Not Sustainable'

In [6]:
SPY_adj_close = si.get_data('SPY', start_date= str(date.today().year - 10) +'-'+ str(date.today().month) +'-'+ str(date.today().day), end_date=str(date.today()))['adjclose']
cummulative_prod_SPY = cumm_prod(SPY_adj_close)
annu_return_10y_SPY = annualized_return(cummulative_prod_SPY, 10)[0]/100

In [7]:
annu_return_10y_SPY

0.140759172300704

In [8]:
for index, row in df.iterrows():

    df['Earnings_expectation'][index] = earnings_expectations(df['pe_forward'][index], df['pe_trailing'][index])

    df['irv_status'][index] = intrinsic_valuation(df['ticker'][index], df['eps_ttm'][index], df['growth_estimate_5y'][index], df['pe_trailing'][index], 0.15)[0]
    df['irv_FairValue'][index] = intrinsic_valuation(df['ticker'][index], df['eps_ttm'][index], df['growth_estimate_5y'][index], df['pe_trailing'][index], 0.15)[1]

    df['capm_status'][index] = CAPM_analysis(df['beta'][index], df['annu_return_10y'][index], annu_return_10y_SPY)[0]
    df['capm_expected_return'][index] = CAPM_analysis(df['beta'][index], df['annu_return_10y'][index], annu_return_10y_SPY)[1]

    df['esg_status'][index] = esg_status(df['ESG_risk'][index])

    df['marketcap_cat'][index] = market_cap_cat(df['market_cap'][index])

    if index % 10 == 0:
        print(str(index) + ' / ' + str(len(df)))



0 / 938
10 / 938
20 / 938
30 / 938
40 / 938
50 / 938
60 / 938
70 / 938
80 / 938
90 / 938
100 / 938
110 / 938
120 / 938
130 / 938
140 / 938
150 / 938
160 / 938
170 / 938
180 / 938
190 / 938
200 / 938
210 / 938
220 / 938
230 / 938
240 / 938
250 / 938
260 / 938
270 / 938
280 / 938
290 / 938
300 / 938
310 / 938
320 / 938
330 / 938
340 / 938
350 / 938
360 / 938
370 / 938
380 / 938
390 / 938
400 / 938
410 / 938
420 / 938
430 / 938
440 / 938
450 / 938
460 / 938
470 / 938
480 / 938
490 / 938
500 / 938
510 / 938
520 / 938
530 / 938
540 / 938
550 / 938
560 / 938
570 / 938
580 / 938
590 / 938
600 / 938
610 / 938
620 / 938
630 / 938
640 / 938
650 / 938
660 / 938
670 / 938
680 / 938
690 / 938
700 / 938
710 / 938
720 / 938
730 / 938
740 / 938
750 / 938
760 / 938
770 / 938
780 / 938
790 / 938
800 / 938
810 / 938
820 / 938
830 / 938
840 / 938
850 / 938
860 / 938
870 / 938
880 / 938
890 / 938
900 / 938
910 / 938
920 / 938
930 / 938


In [9]:
df

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,marketcap_cat,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,Large,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149
1,ABT,Abbott Laboratories,Healthcare,Increase,Above Fair Price,OverValued,Average,118.2900,0.181179,2.091710e+11,Large,0.74,3.940,0.1318,22.924420,30.022842,51.234756,25.96,0.109154
2,ACN,Accenture plc,Technology,Increase,Above Fair Price,OverValued,Sustainable,323.9050,0.208836,2.047083e+11,Large,1.21,9.613,0.1135,31.025385,33.694477,121.148794,9.45,0.166287
3,ADM,Archer-Daniels-Midland Company,Consumer Defensive,Increase,Above Fair Price,OverValued,Not Sustainable,76.9850,0.122815,4.327843e+10,Large,0.81,4.790,0.0660,14.804809,16.072025,19.449588,36.42,0.117663
4,ADP,"Automatic Data Processing, Inc.",Industrials,Increase,Above Fair Price,OverValued,Sustainable,201.0100,0.182708,8.443324e+10,Large,0.83,6.460,0.1371,26.483530,31.116098,90.801919,14.18,0.120094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,XRX,Xerox Holdings Corporation,Technology,Increase,Above Fair Price,UnderValued,No Data,20.6059,0.029031,3.678401e+09,Small,1.75,1.459,-0.1600,10.302950,14.123303,0.609810,999.00,0.231929
934,ZD,"Ziff Davis, Inc.",Communication Services,Increase,Above Fair Price,OverValued,No Data,102.4800,0.145864,4.941668e+09,Small,1.01,4.120,0.1140,14.193907,24.873789,38.485339,999.00,0.141975
935,ZUMZ,Zumiez Inc.,Consumer Cyclical,Decrease,Above Fair Price,UnderValued,No Data,42.6600,0.035214,9.777245e+08,Small,1.68,4.870,0.1500,9.233767,8.759754,21.330001,999.00,0.223419
936,ESNT,Essent Group Ltd.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,44.4800,0.082041,4.912015e+09,Small,1.33,5.569,0.1555,7.339934,7.987071,23.215807,999.00,0.180874


In [10]:
df.to_csv('Final_stock_data.csv', index_label=False)